# 1) Thiết lập đường dẫn và thư mục gốc

In [1]:
from pathlib import Path
import re, shutil, csv
from collections import Counter

# Đường dẫn INPUT_DIR
INPUT_DIR = Path("/kaggle/input/utkface/UTKFace")
OUT_DIR   = Path("/kaggle/working/selected/elderly")
OUT_DIR.mkdir(parents=True, exist_ok=True)

MANIFEST = Path("/kaggle/working/elderly_manifest.csv")


# 2) Lọc ảnh người cao tuổi (≥60) và ghi manifest

In [2]:
# Pattern: age_gender_race_date&time.jpg(.chip.jpg)
pat = re.compile(r"^(?P<age>\d+)_(?P<gender>[01])_(?P<race>\d+)_")

rows = []
total = 0
elderly = 0
age_hist = Counter()

# Duyệt đệ quy toàn bộ ảnh .jpg / .jpg.chip.jpg
for f in INPUT_DIR.rglob("*.jpg*"):
    name = f.name
    m = pat.match(name)
    if not m:
        continue
    total += 1
    age = int(m.group("age"))
    gender = int(m.group("gender"))  # 0=male, 1=female (theo mô tả UTKFace)
    race = int(m.group("race"))

    age_hist[age] += 1

    if age >= 60:
        elderly += 1
        # đặt tên đầu ra giữ nguyên để dễ truy vết
        dst = OUT_DIR / name
        if not dst.exists():
            shutil.copy2(f, dst)
        rows.append({
            "image_id": name,
            "age": age,
            "gender": gender,
            "race": race,
            "path": str(dst)
        })

# Ghi manifest
with MANIFEST.open("w", newline="", encoding="utf-8") as fp:
    w = csv.DictWriter(fp, fieldnames=["image_id","age","gender","race","path"])
    w.writeheader()
    w.writerows(rows)

print(f"Tổng ảnh quét: {total}")
print(f"Ảnh >=60 tuổi đã chọn: {elderly}")
print(f"Manifest: {MANIFEST}")
print(f"Thư mục ảnh đã lọc: {OUT_DIR}")


Tổng ảnh quét: 23705
Ảnh >=60 tuổi đã chọn: 2688
Manifest: /kaggle/working/elderly_manifest.csv
Thư mục ảnh đã lọc: /kaggle/working/selected/elderly


In [3]:
# Hiển thị 6 file đầu tiên sau khi lọc
sample_files = list(OUT_DIR.glob("*.jpg*"))[:6]
sample_files


[PosixPath('/kaggle/working/selected/elderly/69_0_1_20170113184901806.jpg.chip.jpg'),
 PosixPath('/kaggle/working/selected/elderly/66_1_0_20170110140430277.jpg.chip.jpg'),
 PosixPath('/kaggle/working/selected/elderly/71_0_0_20170109015622110.jpg.chip.jpg'),
 PosixPath('/kaggle/working/selected/elderly/65_0_3_20170119205855505.jpg.chip.jpg'),
 PosixPath('/kaggle/working/selected/elderly/70_0_0_20170111200516733.jpg.chip.jpg'),
 PosixPath('/kaggle/working/selected/elderly/60_1_0_20170110143505801.jpg.chip.jpg')]

In [4]:
import shutil

# Zip lại folder elderly
shutil.make_archive("/kaggle/working/elderly_faces", 'zip', "/kaggle/working/selected/elderly")

print("Đã zip xong: /kaggle/working/elderly_faces.zip")


Đã zip xong: /kaggle/working/elderly_faces.zip
